### Agent to Agent Protocol (A2A)

**A2A (Agent-to-Agent)** es un protocolo abierto impulsado por Google que permite la comunicación y colaboración directa entre agentes de inteligencia artificial, sin importar qué empresa, tecnología o plataforma los haya desarrollado. Su objetivo es estandarizar la forma en que los agentes de IA se descubren, intercambian tareas, negocian resultados y coordinan acciones, facilitando la creación de sistemas multiagente interoperables, modulares y seguros.

A2A se basa en tecnologías ampliamente adoptadas como HTTP, JSON-RPC y Server-Sent Events, lo que facilita su integración en infraestructuras existentes. Cada agente publica una “tarjeta de agente” (Agent Card) en formato JSON, donde describe sus capacidades, puntos de contacto y requisitos de autenticación. Así, otros agentes pueden descubrirlo y saber qué tareas puede realizar.

El protocolo gestiona el ciclo de vida de las tareas, permitiendo que un agente delegue trabajos a otros, reciba actualizaciones de estado y comparta resultados en distintos formatos (texto, imágenes, audio, etc.). Además, A2A es seguro por diseño, incorporando mecanismos de autenticación y autorización de nivel empresarial.

En resumen, A2A es la base técnica para que los agentes de IA no solo piensen de forma autónoma, sino que colaboren entre sí, abriendo la puerta a ecosistemas de inteligencia artificial distribuidos y mucho más potentes.

**Los componentes principales del protocolo A2A son:**

- **Tarjeta de Agente** : Es un archivo JSON publicado por cada agente en una ruta estándar. Describe sus capacidades, endpoints, métodos de autenticación y tipos de mensajes soportados. Permite el descubrimiento automático y la negociación de tareas entre agentes.

- **Cliente A2A**: Es el agente que inicia una tarea, actuando como solicitante. Se encarga de identificar qué agentes pueden ejecutar la tarea y de coordinar el flujo de trabajo.

- **Servidor A2A**: Es el agente que recibe la tarea y la ejecuta. Expone endpoints HTTP y anuncia sus capacidades mediante la Tarjeta de Agente.

- **Tarea**: Es la unidad central de trabajo en A2A. Cada tarea tiene un ciclo de vida definido (por ejemplo: enviada, en progreso, completada) e incluye el objetivo, el contexto y los resultados esperados.

- **Mensaje**: Es la comunicación que se intercambia entre cliente y servidor en el contexto de una tarea. Los mensajes pueden contener varias partes con diferentes tipos de contenido.

- **Parte**: Es la unidad mínima de contenido dentro de un mensaje o artefacto. Puede ser texto, archivo, datos estructurados (JSON), etc.

- **Artefacto**: Es el resultado generado por un agente durante la ejecución de una tarea. Puede incluir archivos, fragmentos de conocimiento o cualquier otro tipo de salida relevante.



```mermaid
flowchart LR
    Client[Client]
    ClientAgent[Client Agent]
    subgraph Server["A2A Starlette Application / Server"]
        subgraph Executor["Weather Agent Executor"]
            WeatherAgent["Weather Agent"]
        end
    end

    Client -- "Request\n<span style='font-size:10px'>(Get weather for Miami Florida)</span>" --> ClientAgent
    ClientAgent -- "agent_card" --> Server
    ClientAgent <--> |"RPC"| Server

```



### Hello World A2A

Para arrancar el servidor. Situarse en `27_A2A/a2a_simple` y:

```bash
uv run .
```

Para ejecutar el cliente

```bash
uv run --activate test_client
```

In [4]:
import os
os.makedirs("27_A2A/a2a_simple", exist_ok=True)

In [6]:
%%writefile "27_A2A/a2a_simple/__main__.py"
import uvicorn
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from agent_executor import GreetingAgentExecutor


def main():
    skill = AgentSkill(
        id="hello_world",
        name="Saludar",
        description="Devuelve un saludo",
        tags=["saludo", "hola", "mundo"],
        examples=["Hola", "¿Qué tal?", "Saludos"],
    )

    agent_card = AgentCard(
        name="Agente de Saludo",
        description="Un agente simple que devuelve un saludo",
        url="http://localhost:9999/",
        defaultInputModes=["text"],
        defaultOutputModes=["text"],
        skills=[skill],
        version="1.0.0",
        capabilities=AgentCapabilities(),
    )

    request_handler = DefaultRequestHandler(
        agent_executor=GreetingAgentExecutor(),
        task_store=InMemoryTaskStore(),
    )

    server = A2AStarletteApplication(
        http_handler=request_handler,
        agent_card=agent_card,
    )

    uvicorn.run(server.build(), host="0.0.0.0", port=9999)


if __name__ == "__main__":
    main()

Overwriting 27_A2A/a2a_simple/__main__.py


In [9]:
%%writefile "27_A2A/a2a_simple/agent_executor.py"
from a2a.server.agent_execution import AgentExecutor
from a2a.server.agent_execution.context import RequestContext
from a2a.server.events.event_queue import EventQueue
from a2a.utils import new_agent_text_message
from pydantic import BaseModel


class GreetingAgent(BaseModel):
    """Agente de saludo que devuelve un saludo"""

    async def invoke(self) -> str:
        return "¡Hola YouTube! ¡Asegúrense de darle me gusta y suscribirse!"


class GreetingAgentExecutor(AgentExecutor):

    def __init__(self):
        self.agent = GreetingAgent()

    async def execute(self, context: RequestContext, event_queue: EventQueue):
        result = await self.agent.invoke()
        await event_queue.enqueue_event(new_agent_text_message(result))

    async def cancel(self, context: RequestContext, event_queue: EventQueue):
        raise Exception("Cancelación no soportada")


Overwriting 27_A2A/a2a_simple/agent_executor.py


In [8]:
%%writefile "27_A2A/a2a_simple/test_client.py"
import uuid

import httpx
from a2a.client import A2ACardResolver, A2AClient
from a2a.types import (
    AgentCard,
    Message,
    MessageSendParams,
    Part,
    Role,
    SendMessageRequest,
    TextPart,
)

PUBLIC_AGENT_CARD_PATH = "/.well-known/agent.json"
BASE_URL = "http://localhost:9999"


async def main() -> None:
    async with httpx.AsyncClient() as httpx_client:
        # Inicializar A2ACardResolver
        resolver = A2ACardResolver(
            httpx_client=httpx_client,
            base_url=BASE_URL,
        )

        final_agent_card_to_use: AgentCard | None = None

        try:
            print(
                f"Obteniendo tarjeta de agente pública de: {BASE_URL}{PUBLIC_AGENT_CARD_PATH}"
            )
            _public_card = await resolver.get_agent_card()
            print("Tarjeta de agente pública obtenida")
            print(_public_card.model_dump_json(indent=2))

            final_agent_card_to_use = _public_card

        except Exception as e:
            print(f"Error al obtener la tarjeta de agente pública: {e}")
            raise RuntimeError("Fallo al obtener la tarjeta de agente pública")

        client = A2AClient(
            httpx_client=httpx_client, agent_card=final_agent_card_to_use
        )
        print("A2AClient inicializado")

        message_payload = Message(
            role=Role.user,
            messageId=str(uuid.uuid4()),
            parts=[Part(root=TextPart(text="Hola, ¿cómo estás?"))],
        )
        request = SendMessageRequest(
            id=str(uuid.uuid4()),
            params=MessageSendParams(
                message=message_payload,
            ),
        )
        print("Enviando mensaje")

        response = await client.send_message(request)
        print("Respuesta:")
        print(response.model_dump_json(indent=2))


if __name__ == "__main__":
    import asyncio

    asyncio.run(main())

Writing 27_A2A/a2a_simple/test_client.py


### Friend Scheduling Demo

Ver directorio `27_A2A/a2a_friend_scheduling`.

